In [34]:
# Dataset
from sklearn import datasets# Data processing
import pandas as pd
import numpy as np# Standardize the data
from sklearn.preprocessing import StandardScaler# Model and performance evaluation
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import precision_recall_fscore_support as score# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

In [35]:
import pandas as pd


df =  pd.read_csv(r"C:\Users\Thomas Aujoux\Documents\GitHub\regression\data_feature2.cvs")
df


,Unnamed: 0,playlist_id,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,...,diff_key,diff_loudness,diff_mode,diff_speechiness,diff_acousticness,diff_instrumentalness,diff_liveness,diff_valence,diff_tempo,diff_time_signature
0,0,0,230199.338710,0.694032,0.717952,5.016129,-5.626774,0.467742,0.095531,0.128301,...,0.000000,0.017377,0.000000,-0.000287,0.002197,-1.336066e-06,0.001534,-0.009393,0.507180,0.000000
1,1,1,223173.420455,0.694011,0.705062,5.363636,-5.345648,0.670455,0.121415,0.105880,...,-0.022857,0.002766,0.005714,-0.000033,0.001222,-6.571429e-07,0.000939,0.000126,0.155000,0.000000
2,2,2,217341.061224,0.774286,0.696449,5.173469,-5.906510,0.591837,0.196454,0.101134,...,-0.092784,0.063969,0.010309,0.002124,0.000656,-1.134021e-07,-0.000866,0.003072,0.184649,0.000000
3,3,3,221365.400000,0.749550,0.598450,5.250000,-6.484350,0.550000,0.138900,0.146152,...,0.421053,-0.114158,0.052632,0.004584,-0.004579,0.000000e+00,-0.002368,0.020263,2.716105,0.052632
4,4,4,218697.300000,0.673900,0.697200,6.800000,-6.125600,0.400000,0.169970,0.155313,...,0.444444,-0.176778,0.111111,-0.034400,-0.021067,6.746147e-19,0.005800,-0.016444,-0.585000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,1374,1752,216886.511364,0.661795,0.676364,4.909091,-6.389284,0.431818,0.079250,0.216199,...,0.000000,-0.025402,-0.011494,0.001308,0.002632,-8.735632e-07,0.000507,0.001563,-0.408184,0.000000
1375,1375,1753,230981.700000,0.588500,0.546400,6.500000,-6.747600,0.500000,0.053950,0.120250,...,0.333333,0.230778,0.000000,0.005978,0.022533,0.000000e+00,0.001111,0.048778,-4.553111,0.111111
1376,1376,1754,230694.703704,0.530099,0.708975,4.987654,-6.958000,0.691358,0.052427,0.145611,...,-0.112500,0.008587,0.000000,-0.000152,-0.005063,1.073750e-03,0.000754,0.001512,0.776575,0.000000
1377,1377,1755,214761.453552,0.628311,0.711454,5.387978,-5.662967,0.464481,0.092725,0.101176,...,-0.060440,-0.000593,0.000000,-0.000378,0.000176,6.538462e-08,-0.001066,-0.000418,-0.028308,0.000000


In [36]:
def conditions(s):
    if (s['playlist_id'] < 1000):
        return 1
    else:
        return 0
    
df['Class'] = df.apply(conditions, axis=1)
df
columns = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'diff_duration_ms', 'diff_danceability', 'diff_energy', 'diff_key', 'diff_loudness', 'diff_mode', 'diff_speechiness', 'diff_acousticness', 'diff_instrumentalness', 'diff_liveness', 'diff_valence', 'diff_tempo', 'diff_time_signature']
x = df[columns]
y = df.Class.values

In [37]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(X_train)} records.')
print(f'The testing dataset has {len(X_test)} records.')

The training dataset has 1103 records.
The testing dataset has 276 records.


In [38]:
# Initiate scaler
sc = StandardScaler()# Standardize the training dataset
X_train_transformed = pd.DataFrame(sc.fit_transform(X_train),index=X_train.index, columns=X_train.columns)# Standardized the testing dataset
X_test_transformed = pd.DataFrame(sc.transform(X_test),index=X_test.index, columns=X_test.columns)# Summary statistics after standardization
X_train_transformed.describe().T

,count,mean,std,min,25%,50%,75%,max
duration_ms,1103.0,-1.046810e-15,1.000454,-5.888050,-0.552444,-0.114118,0.411793,7.163295
danceability,1103.0,-1.327839e-15,1.000454,-6.119221,-0.729181,0.011320,0.745484,2.238345
energy,1103.0,1.049226e-15,1.000454,-4.403438,-0.592759,0.092696,0.757590,2.601048
key,1103.0,-4.992481e-17,1.000454,-4.310056,-0.560246,-0.028772,0.549247,4.746507
loudness,1103.0,-1.288382e-16,1.000454,-9.490253,-0.499035,0.195816,0.712113,2.177437
mode,1103.0,4.106718e-16,1.000454,-2.892051,-0.666650,-0.110432,0.634515,2.700238
speechiness,1103.0,4.831433e-18,1.000454,-1.395456,-0.818393,-0.275788,0.709551,4.846949
acousticness,1103.0,8.052388e-18,1.000454,-1.473396,-0.749797,-0.337095,0.494551,4.721202
instrumentalness,1103.0,4.831433e-18,1.000454,-0.527956,-0.494539,-0.371976,-0.012918,8.330632
liveness,1103.0,4.799223e-16,1.000454,-2.544754,-0.693413,-0.067684,0.556763,7.428548


In [42]:
# Train the model
xgboost = XGBRegressor(seed=0).fit(X_train_transformed,y_train)# Make prediction
xgboost_predict = xgboost.predict(X_test_transformed)# Get predicted probability
xgboost_predict_prob = xgboost.predict(X_test)

In [47]:
from sklearn.metrics import mean_squared_error
# Evaluate the model
print('MSE: ', mean_squared_error(y_test, xgboost_predict))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, xgboost_predict)))


MSE:  0.14015447219567656
RMSE:  0.3743721039229239


In [44]:
# Get performance metrics
precision, recall, fscore, support = score(y_test, xgboost_predict)# Print result
print(f'The recall value for the baseline xgboost model is {recall:.4f}')

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [14]:
# Define the search space
param_grid = { 
    # Percentage of columns to be randomly samples for each tree.
    "colsample_bytree": [ 0.3, 0.5 , 0.8 ],
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5, 1, 5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5, 1, 5]
    }# Set up score
scoring = ['recall']# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Define grid search
grid_search = GridSearchCV(estimator=xgboost, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='recall', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)# Fit grid search
grid_result = grid_search.fit(X_train_transformed, y_train)# Print grid search summary
grid_result# Print the best score and the corresponding hyperparameters
print(f'The best score is {grid_result.best_score_:.4f}')
print('The best score standard deviation is', round(grid_result.cv_results_['std_test_recall'][grid_result.best_index_], 4))
print(f'The best hyperparameters are {grid_result.best_params_}')

The best score is 0.8755
The best score standard deviation is 0.025
The best hyperparameters are {'colsample_bytree': 0.8, 'reg_alpha': 5, 'reg_lambda': 0}


In [15]:
# Make prediction using the best model
grid_predict = grid_search.predict(X_test_transformed)# Get predicted probabilities
grid_predict_prob = grid_search.predict_proba(X_test_transformed)[:,1]# Get performance metrics
precision, recall, fscore, support = score(y_test_transformed, grid_predict)# Print result
print(f'The recall value for the xgboost grid search is {recall[1]:.4f}')

NameError: name 'y_test_transformed' is not defined

In [16]:
# Define the search space
param_grid = { 
    # Learning rate shrinks the weights to make the boosting process more conservative
    "learning_rate": [0.0001,0.001, 0.01, 0.1, 1] ,
    # Maximum depth of the tree, increasing it increases the model complexity.
    "max_depth": range(3,21,3),
    # Gamma specifies the minimum loss reduction required to make a split.
    "gamma": [i/10.0 for i in range(0,5)],
    # Percentage of columns to be randomly samples for each tree.
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]}# Set up score
scoring = ['recall']# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [17]:
# Define random search
random_search = RandomizedSearchCV(estimator=xgboost, 
                           param_distributions=param_grid, 
                           n_iter=48,
                           scoring=scoring, 
                           refit='recall', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)# Fit grid search
random_result = random_search.fit(X_train_transformed, y_train)# Print grid search summary
random_result# Print the best score and the corresponding hyperparameters
print(f'The best score is {random_result.best_score_:.4f}')
print('The best score standard deviation is', round(random_result.cv_results_['std_test_recall'][random_result.best_index_], 4))
print(f'The best hyperparameters are {random_result.best_params_}')

The best score is 0.8755
The best score standard deviation is 0.0173
The best hyperparameters are {'reg_lambda': 1, 'reg_alpha': 1, 'max_depth': 6, 'learning_rate': 0.0001, 'gamma': 0.0, 'colsample_bytree': 0.4}


In [18]:
# Make prediction using the best model
random_predict = random_search.predict(X_test_transformed)# Get predicted probabilities
random_predict_prob = random_search.predict_proba(X_test_transformed)[:,1]# Get performance metrics
precision, recall, fscore, support = score(y_test, random_predict)# Print result
print(f'The recall value for the xgboost random search is {recall[1]:.4f}')

The recall value for the xgboost random search is 0.8205


In [19]:
# Space
space = {
    'learning_rate': hp.choice('learning_rate', [0.0001,0.001, 0.01, 0.1, 1]),
    'max_depth' : hp.choice('max_depth', range(3,21,3)),
    'gamma' : hp.choice('gamma', [i/10.0 for i in range(0,5)]),
    'colsample_bytree' : hp.choice('colsample_bytree', [i/10.0 for i in range(3,10)]),     
    'reg_alpha' : hp.choice('reg_alpha', [1e-5, 1e-2, 0.1, 1, 10, 100]), 
    'reg_lambda' : hp.choice('reg_lambda', [1e-5, 1e-2, 0.1, 1, 10, 100])
}

In [20]:
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [22]:
# Objective function
def objective(params):
    
    xgboost = XGBClassifier(seed=0, **params)
    score = cross_val_score(estimator=xgboost, 
                            X=X_train_transformed, 
                            y=y_train, 
                            cv=kfold, 
                            scoring='recall', 
                            n_jobs=-1).mean()
    # Loss is negative score
    loss = - score    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [23]:
# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 48, trials = Trials())

100%|██████████| 48/48 [00:14<00:00,  3.23trial/s, best loss: -0.8735441908518832]


In [24]:
# Print the index of the best parameters
print(best)
# Print the values of the best parameters
print(space_eval(space, best))

{'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 2, 'max_depth': 4, 'reg_alpha': 2, 'reg_lambda': 4}
{'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.01, 'max_depth': 15, 'reg_alpha': 0.1, 'reg_lambda': 10}


In [25]:
# Train model using the best parameters
xgboost_bo = XGBClassifier(seed=0, 
                           colsample_bytree=space_eval(space, best)['colsample_bytree'], 
                           gamma=space_eval(space, best)['gamma'], 
                           learning_rate=space_eval(space, best)['learning_rate'], 
                           max_depth=space_eval(space, best)['max_depth'], 
                           reg_alpha=space_eval(space, best)['reg_alpha'],
                           reg_lambda=space_eval(space, best)['reg_lambda']
                           ).fit(X_train_transformed,y_train)

In [26]:
#Make prediction using the best model
bayesian_opt_predict = xgboost_bo.predict(X_test_transformed)# Get predicted probabilities
bayesian_opt_predict_prob = xgboost_bo.predict_proba(X_test_transformed)[:,1]# Get performance metrics
precision, recall, fscore, support = score(y_test, bayesian_opt_predict)# Print result
print(f'The recall value for the xgboost Bayesian optimization is {recall[1]:.4f}')

The recall value for the xgboost Bayesian optimization is 0.8205


In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [50]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(X_train)} records.')
print(f'The testing dataset has {len(X_test)} records.')

The training dataset has 1103 records.
The testing dataset has 276 records.


In [51]:
gbm = xgb.XGBRegressor()
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                            ,'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=1)
reg_cv.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'colsample_bytree': [1.0], 'max_depth': [3, 4, 6],
                         'min_child_weight': [1.0, 1.2],
                         'n_estimators': [500, 1000]},
             verbose=1)

In [52]:


reg_cv.best_params_



{'colsample_bytree': 1.0,
 'max_depth': 3,
 'min_child_weight': 1.0,
 'n_estimators': 500}

In [53]:
gbm = xgb.XGBRegressor(**reg_cv.best_params_)
gbm.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1.0, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [54]:
predictions = gbm.predict(X_test)
predictions


array([-9.92301852e-02,  3.91360700e-01,  4.09737006e-02,  9.73181903e-01,
       -8.63957107e-02,  4.44442369e-02,  5.51411927e-01,  1.30132437e-02,
        6.88879311e-01,  6.57675564e-01,  1.66097656e-01,  7.78142691e-01,
       -7.57554099e-02, -6.37866482e-02, -4.33795862e-02,  1.07736506e-01,
        8.28918755e-01,  6.08600080e-01, -7.92823453e-03,  8.42894375e-01,
        1.37399687e-02,  2.95409471e-01,  5.40433466e-01,  1.93351850e-01,
        7.33669221e-01, -1.38491467e-02,  9.00721475e-02, -5.27575687e-02,
        8.13615143e-01,  9.06094372e-01,  8.17069411e-01, -3.47769290e-01,
        2.78960407e-01, -5.24804704e-02,  1.41552597e-01,  5.33233248e-02,
        3.97083797e-02,  1.07068086e+00, -5.89990243e-02, -1.36292413e-01,
       -3.33531275e-02, -3.51692922e-03,  2.74349362e-01,  1.03500140e+00,
        1.19498195e-02,  1.01835847e+00, -1.01941384e-01,  5.09548247e-01,
        1.04933453e+00,  7.39846170e-01, -9.72324163e-02,  1.84746549e-01,
        1.00315964e+00, -

In [55]:
gbm.score(X_test,y_test)

0.4725426586995043

In [56]:
gbm.score(X_train,y_train)

0.9977941711314783